In [1]:
import pandas as pd # for dataframes
import matplotlib.pyplot as plt # for plotting graphs
import seaborn as sns # for plotting graphs
import datetime as dt
from datetime import date, time, datetime
from operator import attrgetter

In [19]:
data = pd.read_excel('ecom_data.xlsx', sheet_name=['Orders','Customers','Sales_people','Returns'])

C:\Users\Bogdan\anaconda3\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
C:\Users\Bogdan\anaconda3\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
C:\Users\Bogdan\anaconda3\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [20]:
orders = data['Orders']
cutomers = data['Customers']
sales_people = data['Sales_people']
returns = data['Returns']

In [23]:
orders.head()

,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Country/Region,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,CA-2019-152156,2019-11-08,2019-11-11,Second Class,CG-12520,United States,Henderson,Kentucky,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,CA-2019-152156,2019-11-08,2019-11-11,Second Class,CG-12520,United States,Henderson,Kentucky,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,CA-2019-138688,2019-06-12,2019-06-16,Second Class,DV-13045,United States,Los Angeles,California,90036.0,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,US-2018-108966,2018-10-11,2018-10-18,Standard Class,SO-20335,United States,Fort Lauderdale,Florida,33311.0,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,US-2018-108966,2018-10-11,2018-10-18,Standard Class,SO-20335,United States,Fort Lauderdale,Florida,33311.0,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [26]:
orders.dropna()

,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Country/Region,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,CA-2019-152156,2019-11-08,2019-11-11,Second Class,CG-12520,United States,Henderson,Kentucky,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,CA-2019-152156,2019-11-08,2019-11-11,Second Class,CG-12520,United States,Henderson,Kentucky,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,CA-2019-138688,2019-06-12,2019-06-16,Second Class,DV-13045,United States,Los Angeles,California,90036.0,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,US-2018-108966,2018-10-11,2018-10-18,Standard Class,SO-20335,United States,Fort Lauderdale,Florida,33311.0,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,US-2018-108966,2018-10-11,2018-10-18,Standard Class,SO-20335,United States,Fort Lauderdale,Florida,33311.0,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9989,CA-2017-110422,2017-01-21,2017-01-23,Second Class,TB-21400,United States,Miami,Florida,33180.0,South,FUR-FU-10001889,Furniture,Furnishings,Ultra Door Pull Handle,25.2480,3,0.20,4.1028
9990,CA-2020-121258,2020-02-26,2020-03-03,Standard Class,DB-13060,United States,Costa Mesa,California,92627.0,West,FUR-FU-10000747,Furniture,Furnishings,Tenex B1-RE Series Chair Mats for Low Pile Car...,91.9600,2,0.00,15.6332
9991,CA-2020-121258,2020-02-26,2020-03-03,Standard Class,DB-13060,United States,Costa Mesa,California,92627.0,West,TEC-PH-10003645,Technology,Phones,Aastra 57i VoIP phone,258.5760,2,0.20,19.3932
9992,CA-2020-121258,2020-02-26,2020-03-03,Standard Class,DB-13060,United States,Costa Mesa,California,92627.0,West,OFF-PA-10004041,Office Supplies,Paper,"It's Hot Message Books with Stickers, 2 3/4"" x 5""",29.6000,4,0.00,13.3200


In [27]:
orders.describe()

,Order Date,Ship Date,Postal Code,Sales,Quantity,Discount,Profit
count,9994,9994,9983.000000,9994.000000,9994.000000,9994.000000,9994.000000
mean,2019-04-30 17:41:20.448268800,2019-05-04 16:42:15.921553152,55245.233297,229.858001,3.789574,0.156203,28.656896
min,2017-01-03 00:00:00,2017-01-07 00:00:00,1040.000000,0.444000,1.000000,0.000000,-6599.978000
25%,2018-05-23 00:00:00,2018-05-27 00:00:00,23223.000000,17.280000,2.000000,0.000000,1.728750
50%,2019-06-26 00:00:00,2019-06-29 00:00:00,57103.000000,54.490000,3.000000,0.200000,8.666500
75%,2020-05-14 00:00:00,2020-05-18 00:00:00,90008.000000,209.940000,5.000000,0.200000,29.364000
max,2020-12-30 00:00:00,2021-01-05 00:00:00,99301.000000,22638.480000,14.000000,0.800000,8399.976000
std,NaN,NaN,32038.715955,623.245101,2.225110,0.206452,234.260108


In [28]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Order ID        9994 non-null   object        
 1   Order Date      9994 non-null   datetime64[ns]
 2   Ship Date       9994 non-null   datetime64[ns]
 3   Ship Mode       9994 non-null   object        
 4   Customer ID     9994 non-null   object        
 5   Country/Region  9994 non-null   object        
 6   City            9994 non-null   object        
 7   State           9994 non-null   object        
 8   Postal Code     9983 non-null   float64       
 9   Region          9994 non-null   object        
 10  Product ID      9994 non-null   object        
 11  Category        9994 non-null   object        
 12  Sub-Category    9994 non-null   object        
 13  Product Name    9994 non-null   object        
 14  Sales           9994 non-null   float64       
 15  Quan